In [1]:
!pip install cassandra-driver

In [5]:
from json import loads
import cassandra
cassandra. __version__

'3.25.0'

In [7]:
from cassandra.cluster import Cluster
from elasticsearch import Elasticsearch

cluster = Cluster()
session = cluster.connect()
session.execute("CREATE KEYSPACE IF NOT EXISTS mydb WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 };")

elasticSearch = Elasticsearch([{'host':'localhost','port':9200}])

In [8]:
from kafka import KafkaConsumer
from kafka import KafkaProducer
import time
import re
import json
from json import loads
from time import sleep
from json import dumps
import requests

consumer = KafkaConsumer(
    'persistence',
     bootstrap_servers=['192.168.1.3:9092'],
     auto_offset_reset='earliest',
     enable_auto_commit=True,
     group_id='my-group3',
     api_version=(0,10),
     value_deserializer=lambda x: loads(x.decode('utf-8')))

producer = KafkaProducer(bootstrap_servers=['192.168.1.3:9092'],key_serializer=str.encode,api_version=(0,10),
                         value_serializer=lambda x:dumps(x).encode('utf-8'))

# Make a Dataframe to store the consumer contents:

In [9]:
import pandas as pd
df=pd.DataFrame(columns=['hashtag','id','year', 'month','day','hour','minute'])
df

,hashtag,id,year,month,day,hour,minute


In [10]:
import datetime

for message in consumer:
    message = message.value
    if 'content' not in message:
        continue
    
    #Inserting data to Elastic Search:
    try:
        elasticSearch.index(index='twitter_bors',doc_type='twitt',body = message, id =message['id'])
    except:
        print("An exception occurred")
    
    
    #At the same time, inserting data to data frame to be injected to Cassandra tables:
    id=int(message['id'])
    content=message['content']
    date = datetime.datetime.strptime(message['sendTimePersian'], "%Y/%m/%d  %H:%M")
    year=date.year
    month=date.month
    day=date.day
    hour=date.hour
    minute=date.minute
    
    for h in message['hashtags']:
        new_row={'hashtag':h,'id':id,'year':year,'month':month,'day':day,'hour':hour,'minute':minute,}
        df=df.append(new_row, ignore_index=True)
        
    producer.send('statistics', key=message['id'],value=message)


c:\users\alija\miniconda3\lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: [types removal] Specifying types in document index requests is deprecated, use the typeless endpoints instead (/{index}/_doc/{id}, /{index}/_doc, or /{index}/_create/{id}).
  warnings.warn(message, category=ElasticsearchWarning)


KeyboardInterrupt: 

In [11]:
df

,hashtag,id,year,month,day,hour,minute
0,شاخص_بورس,327959859,1400,4,28,14,43
1,فروی,327959717,1400,4,28,14,41
2,کاما,327959709,1400,4,28,14,41
3,نوین,327959667,1400,4,28,14,41
4,خاور,327959661,1400,4,28,14,41
...,...,...,...,...,...,...,...
101,خپارس,327960374,1400,4,28,14,45
102,سپید,327960329,1400,4,28,14,45
103,وخارزم,327960322,1400,4,28,14,45
104,جم,327960318,1400,4,28,14,45


In [10]:
df[df.iloc[:,0]=='شپلی']

,hashtag,id,year,month,day,hour,minute
1,شپلی,327955297,1400,4,28,14,9
6,شپلی,327955200,1400,4,28,14,8
11,شپلی,327955297,1400,4,28,14,9
16,شپلی,327955200,1400,4,28,14,8
19,شپلی,327955297,1400,4,28,14,9
24,شپلی,327955200,1400,4,28,14,8
27,شپلی,327955297,1400,4,28,14,9
32,شپلی,327955200,1400,4,28,14,8
35,شپلی,327955297,1400,4,28,14,9
40,شپلی,327955200,1400,4,28,14,8


# Elastic Search Sample Queries

In [12]:
res = elasticSearch.get(index="twitter_bors", id='327960279')
print(res['_source'])

{'id': '327960279', 'sendTime': '2021-07-19T10:15:22Z', 'sendTimePersian': '1400/04/28 14:45', 'senderName': 'اماتور  موفق', 'senderUsername': 'amin_invesment', 'senderProfileImage': 'default', 'content': '#خاور \nنمیشه که حد اقل در دو سال گذشته تمام سهام های خودرویی هم از تجهیزات هم از ساختمان و  هم از زمین افزایش سرمایه بدن اما خاور بخش تجهیزاتش حذف بشه حداقل باید گفت نهایت کم لطفی است در حق سهام داران خرد\n و اگر با بد بینی نگاه کنیم برنامه هماهنگ به نفع حقوقی ها شرکت ایرانخودرو و تلیاردر ها است  با رنگ و لعاب قانون', 'type': 'twit', 'scoredPostDate': '1626689785435', 'finalPullDatePersian': '', 'hashtags': ['خاور'], 'links': [], 'keywords': ['خاور', 'حقوقی\u200cها', 'حق', 'حذف', 'تلیاردر\u200cها', 'تجهیزاتش', 'تجهیزات', 'بینی', 'برنامه', 'بدن', 'بد', 'ایرانخودرو', 'اقل', 'گذشته']}


In [13]:
res = elasticSearch.search(index="twitter_bors", body={"query": {"ids": { 'values': [327957732,327957755]}}})
print("Got %d Hits:" % res['hits']['total']['value'])
for hit in res['hits']['hits']:
    print(hit['_source'])

Got 2 Hits:
{'id': '327957755', 'sendTime': '2021-07-19T09:57:45Z', 'sendTimePersian': '1400/04/28 14:27', 'senderName': 'shirkohb', 'senderUsername': 'shirkohb', 'senderProfileImage': 'af832b29-7649-4923-90ab-95444fd7acc0', 'content': '#خودکفا \nبورس شده گدا خونه ، یه عده گدا گشنه با چندرغاز پول اومدن اینجا ، شب و روز بیست و چهار ساعت گله ، طعنه ، بدو بیراه گفتن به این و اون ، همینه عزیز من بازار ، شما که دو تومن تو جیبات نیس غلط کردی اومدی اینجا ، شرکت با این همه عظمت و تشکیلات هیچ کاری نداره بیاد به امثال شماها جواب بده که چرا پونصد هزار تومنت نشده یه تومن ... برو عزیز من اینجا خودتو اذیت نکن کارخونه پولسازی نیس که ... برو بساط دس فروشی را بنداز بیشتر سود می\u200cکنی ... دیگه نه خودتو اذیت کن نه دیگران و ... اینجا کسانی هستند پانزده بیست ساله تو بورس نمیان حتی تحلیلم بزارن ... آدم یه زره فکر کنه میبینه رسیدن به هر چیزی زمان میخواد ... صبر نداری برو ماریو که بهت گفتم و انجام بده ...', 'type': 'twit', 'scoredPostDate': '1626688714007', 'finalPullDatePersian': '', 'hashtags': ['خودکفا'

# تمام پست های مربوط به یک هشتگ

In [14]:
query="CREATE TABLE IF NOT EXISTS mydb.hashtag(\
hashtag text,\
id int,\
PRIMARY KEY (hashtag, id)\
);"

session.execute(query)

In [15]:
session.execute("TRUNCATE mydb.hashtag;")

In [16]:
import pandas as pd
import numpy as np
from cassandra.query import BatchStatement
from cassandra import ConsistencyLevel

ps = session.prepare("INSERT INTO mydb.hashtag (hashtag,id) VALUES (?, ?)")
batch = BatchStatement(consistency_level=ConsistencyLevel.QUORUM)

for i in range(np.int32(np.floor(df.shape[0]/100))):
    batch = BatchStatement(consistency_level=ConsistencyLevel.QUORUM)
    for j in range(100):
        batch.add(ps,(df.iloc[i*100+j,0], df.iloc[i*100+j,1]))
    session.execute(batch)

batch = BatchStatement(consistency_level=ConsistencyLevel.QUORUM)
for i in range(np.int32(np.floor(df.shape[0]/100))*100,df.shape[0]):
    batch.add(ps,(df.iloc[i,0], df.iloc[i,1]))
    
session.execute(batch)

In [17]:
result=session.execute("select * from mydb.hashtag ")
for row in result:
    print(row)

Row(hashtag='شپلی', id=327960011)
Row(hashtag='غسالم', id=327960387)
Row(hashtag='فروی', id=327959717)
Row(hashtag='سپید', id=327960329)
Row(hashtag='برکت', id=327959597)
Row(hashtag='نوین', id=327959667)
Row(hashtag='خپارس', id=327960374)
Row(hashtag='شاخص_بورس', id=327959859)
Row(hashtag='شاخص_بورس', id=327960124)
Row(hashtag='غشهداب', id=327960132)
Row(hashtag='کاما', id=327959709)
Row(hashtag='جم', id=327960318)
Row(hashtag='وخارزم', id=327960322)
Row(hashtag='خودرو', id=327959563)
Row(hashtag='خودرو', id=327959981)
Row(hashtag='خاور', id=327959661)
Row(hashtag='خاور', id=327960279)
Row(hashtag='دی', id=327960164)


In [18]:
result=session.execute("select * from mydb.hashtag where hashtag='خودرو'") #
ids_list=[]
for row in result:
    ids_list.append(row.id)
    print(row.id)

327959563
327959981


In [19]:
res = elasticSearch.search(index="twitter_bors", body={"query": {"ids": { 'values': ids_list}}})
print("Got %d Hits:" % res['hits']['total']['value'])
for hit in res['hits']['hits']:
    print(hit['_source'])

Got 2 Hits:
{'id': '327959563', 'sendTime': '2021-07-19T10:10:24Z', 'sendTimePersian': '1400/04/28 14:40', 'senderName': 'roham', 'senderUsername': 'ramgol', 'senderProfileImage': '4e660afd-ea12-44ad-9d44-f7bd555d0bc6', 'content': '#خودرو با قیمت حدود ۲۵۰ باز میشه و بعدش تا حدود ۱۶۰ تومن ریزش داره مجددا رشد اما آهسته و پیوسته', 'type': 'twit', 'scoredPostDate': '1626689424655', 'finalPullDatePersian': '', 'hashtags': ['خودرو'], 'links': [], 'keywords': ['مجددا', 'پیوسته', '۱۶۰', '۲۵۰']}
{'id': '327959981', 'sendTime': '2021-07-19T10:13:52Z', 'sendTimePersian': '1400/04/28 14:43', 'senderName': 'roham', 'senderUsername': 'ramgol', 'senderProfileImage': '4e660afd-ea12-44ad-9d44-f7bd555d0bc6', 'content': '#خودرو بورس تا دوشنبه آینده تعطیل شد....', 'type': 'twit', 'scoredPostDate': '1626689625994', 'finalPullDatePersian': '', 'hashtags': ['خودرو'], 'links': [], 'keywords': ['دوشنبه', '…', 'بورس']}


In [40]:
ids_list

[327957732]

# تمام پست های مربوط به یک هشتگ در یک بازه زمانی خاص

In [20]:
query="CREATE TABLE IF NOT EXISTS mydb.hashtag_interval(\
hashtag text,\
day int,\
hour int,\
minute int,\
id int,\
PRIMARY KEY (hashtag, day, hour, minute, id)\
);"

session.execute(query)

In [21]:
session.execute("TRUNCATE mydb.hashtag_interval;")

In [22]:
import pandas as pd
import numpy as np
from cassandra.query import BatchStatement
from cassandra import ConsistencyLevel

ps = session.prepare("INSERT INTO mydb.hashtag_interval (hashtag, day, hour, minute, id) VALUES (?, ?, ?, ?, ?)")
batch = BatchStatement(consistency_level=ConsistencyLevel.QUORUM)

for i in range(np.int32(np.floor(df.shape[0]/100))):
    batch = BatchStatement(consistency_level=ConsistencyLevel.QUORUM)
    for j in range(100):
        batch.add(ps,(df.iloc[i*100+j,0], df.iloc[i*100+j,4], df.iloc[i*100+j,5], df.iloc[i*100+j,6], df.iloc[i*100+j,1]))
    session.execute(batch)

batch = BatchStatement(consistency_level=ConsistencyLevel.QUORUM)
for i in range(np.int32(np.floor(df.shape[0]/100))*100,df.shape[0]):
    batch.add(ps,(df.iloc[i,0], df.iloc[i,4], df.iloc[i,5], df.iloc[i,6], df.iloc[i,1]))
    
session.execute(batch)

In [23]:
result=session.execute("select * from mydb.hashtag_interval ") #where hashtag='شپلی'
for row in result:
    print(row)

Row(hashtag='شپلی', day=28, hour=14, minute=44, id=327960011)
Row(hashtag='غسالم', day=28, hour=14, minute=46, id=327960387)
Row(hashtag='فروی', day=28, hour=14, minute=41, id=327959717)
Row(hashtag='سپید', day=28, hour=14, minute=45, id=327960329)
Row(hashtag='برکت', day=28, hour=14, minute=40, id=327959597)
Row(hashtag='نوین', day=28, hour=14, minute=41, id=327959667)
Row(hashtag='خپارس', day=28, hour=14, minute=45, id=327960374)
Row(hashtag='شاخص_بورس', day=28, hour=14, minute=43, id=327959859)
Row(hashtag='شاخص_بورس', day=28, hour=14, minute=44, id=327960124)
Row(hashtag='غشهداب', day=28, hour=14, minute=44, id=327960132)
Row(hashtag='کاما', day=28, hour=14, minute=41, id=327959709)
Row(hashtag='جم', day=28, hour=14, minute=45, id=327960318)
Row(hashtag='وخارزم', day=28, hour=14, minute=45, id=327960322)
Row(hashtag='خودرو', day=28, hour=14, minute=40, id=327959563)
Row(hashtag='خودرو', day=28, hour=14, minute=43, id=327959981)
Row(hashtag='خاور', day=28, hour=14, minute=41, id=327

In [27]:
result=session.execute("select * from mydb.hashtag_interval where hashtag='خودرو' and day=28 and hour=14 and minute>41") #
ids_list=[]
for row in result:
    ids_list.append(row.id)
    print(row)

Row(hashtag='خودرو', day=28, hour=14, minute=43, id=327959981)


In [28]:
res = elasticSearch.search(index="twitter_bors", body={"query": {"ids": { 'values': ids_list}}})
print("Got %d Hits:" % res['hits']['total']['value'])
for hit in res['hits']['hits']:
    print(hit['_source'])

Got 1 Hits:
{'id': '327959981', 'sendTime': '2021-07-19T10:13:52Z', 'sendTimePersian': '1400/04/28 14:43', 'senderName': 'roham', 'senderUsername': 'ramgol', 'senderProfileImage': '4e660afd-ea12-44ad-9d44-f7bd555d0bc6', 'content': '#خودرو بورس تا دوشنبه آینده تعطیل شد....', 'type': 'twit', 'scoredPostDate': '1626689625994', 'finalPullDatePersian': '', 'hashtags': ['خودرو'], 'links': [], 'keywords': ['دوشنبه', '…', 'بورس']}


# تمام پست ها در یک بازه زمانی خاص

In [29]:
query="CREATE TABLE IF NOT EXISTS mydb.total_interval(\
year int,\
month int,\
day int,\
hour int,\
minute int,\
id int,\
PRIMARY KEY ((year, month), day, hour, minute, id)\
);"

session.execute(query)

In [30]:
session.execute("TRUNCATE mydb.total_interval;")
# session.execute("drop table mydb.total_interval;")

In [104]:
df.head()

,hashtag,id,year,month,day,hour,minute
0,شلرد,321482732,1400,4,19,16,5
1,فاهواز,321482718,1400,4,19,16,5
2,شکف,321482707,1400,4,19,16,5
3,سیدکو,321482677,1400,4,19,16,4
4,آینده,321482618,1400,4,19,16,4


In [31]:
import pandas as pd
import numpy as np
from cassandra.query import BatchStatement
from cassandra import ConsistencyLevel

ps = session.prepare("INSERT INTO mydb.total_interval (year, month, day, hour, minute, id) VALUES (?, ?, ?, ?, ?, ?)")
batch = BatchStatement(consistency_level=ConsistencyLevel.QUORUM)

for i in range(np.int32(np.floor(df.shape[0]/100))):
    batch = BatchStatement(consistency_level=ConsistencyLevel.QUORUM)
    for j in range(100):
        batch.add(ps,(df.iloc[i*100+j,2], df.iloc[i*100+j,3], df.iloc[i*100+j,4], df.iloc[i*100+j,5], df.iloc[i*100+j,6], df.iloc[i*100+j,1]))
    session.execute(batch)

batch = BatchStatement(consistency_level=ConsistencyLevel.QUORUM)
for i in range(np.int32(np.floor(df.shape[0]/100))*100,df.shape[0]):
    batch.add(ps,(df.iloc[i,2], df.iloc[i,3], df.iloc[i,4], df.iloc[i,5], df.iloc[i,6], df.iloc[i,1]))
    
session.execute(batch)

In [32]:
result=session.execute("select * from mydb.total_interval ") #where hashtag='شپلی'
for row in result:
    print(row)

Row(year=1400, month=4, day=28, hour=14, minute=40, id=327959563)
Row(year=1400, month=4, day=28, hour=14, minute=40, id=327959597)
Row(year=1400, month=4, day=28, hour=14, minute=41, id=327959661)
Row(year=1400, month=4, day=28, hour=14, minute=41, id=327959667)
Row(year=1400, month=4, day=28, hour=14, minute=41, id=327959709)
Row(year=1400, month=4, day=28, hour=14, minute=41, id=327959717)
Row(year=1400, month=4, day=28, hour=14, minute=43, id=327959859)
Row(year=1400, month=4, day=28, hour=14, minute=43, id=327959981)
Row(year=1400, month=4, day=28, hour=14, minute=44, id=327960011)
Row(year=1400, month=4, day=28, hour=14, minute=44, id=327960124)
Row(year=1400, month=4, day=28, hour=14, minute=44, id=327960132)
Row(year=1400, month=4, day=28, hour=14, minute=45, id=327960164)
Row(year=1400, month=4, day=28, hour=14, minute=45, id=327960279)
Row(year=1400, month=4, day=28, hour=14, minute=45, id=327960318)
Row(year=1400, month=4, day=28, hour=14, minute=45, id=327960322)
Row(year=1

In [33]:
result=session.execute("select * from mydb.total_interval where year=1400 and month=4 ") #
ids_list=[]
for row in result:
    ids_list.append(row.id)
    print(row)

Row(year=1400, month=4, day=28, hour=14, minute=40, id=327959563)
Row(year=1400, month=4, day=28, hour=14, minute=40, id=327959597)
Row(year=1400, month=4, day=28, hour=14, minute=41, id=327959661)
Row(year=1400, month=4, day=28, hour=14, minute=41, id=327959667)
Row(year=1400, month=4, day=28, hour=14, minute=41, id=327959709)
Row(year=1400, month=4, day=28, hour=14, minute=41, id=327959717)
Row(year=1400, month=4, day=28, hour=14, minute=43, id=327959859)
Row(year=1400, month=4, day=28, hour=14, minute=43, id=327959981)
Row(year=1400, month=4, day=28, hour=14, minute=44, id=327960011)
Row(year=1400, month=4, day=28, hour=14, minute=44, id=327960124)
Row(year=1400, month=4, day=28, hour=14, minute=44, id=327960132)
Row(year=1400, month=4, day=28, hour=14, minute=45, id=327960164)
Row(year=1400, month=4, day=28, hour=14, minute=45, id=327960279)
Row(year=1400, month=4, day=28, hour=14, minute=45, id=327960318)
Row(year=1400, month=4, day=28, hour=14, minute=45, id=327960322)
Row(year=1

In [34]:
res = elasticSearch.search(index="twitter_bors", body={"query": {"ids": { 'values': ids_list}}})
print("Got %d Hits:" % res['hits']['total']['value'])
for hit in res['hits']['hits']:
    print(hit['_source'])

Got 18 Hits:
{'id': '327959597', 'sendTime': '2021-07-19T10:10:38Z', 'sendTimePersian': '1400/04/28 14:40', 'senderName': 'Yass', 'senderUsername': 'myass', 'senderProfileImage': 'default', 'content': '#برکت نمادهای برتر ورود و خروج پول\u200cحقیقی در تاریخ ۲۸ تیرماه', 'type': 'twit', 'imageUid': '09823731-bf7a-4022-ab25-5d52818ac067', 'mediaContentType': 'image/jpeg', 'scoredPostDate': '1626689515293', 'finalPullDatePersian': '', 'hashtags': ['برکت'], 'links': [], 'keywords': ['نمادهای', 'ورود', 'پول\u200cحقیقی', '۲۸']}
{'id': '327959563', 'sendTime': '2021-07-19T10:10:24Z', 'sendTimePersian': '1400/04/28 14:40', 'senderName': 'roham', 'senderUsername': 'ramgol', 'senderProfileImage': '4e660afd-ea12-44ad-9d44-f7bd555d0bc6', 'content': '#خودرو با قیمت حدود ۲۵۰ باز میشه و بعدش تا حدود ۱۶۰ تومن ریزش داره مجددا رشد اما آهسته و پیوسته', 'type': 'twit', 'scoredPostDate': '1626689424655', 'finalPullDatePersian': '', 'hashtags': ['خودرو'], 'links': [], 'keywords': ['مجددا', 'پیوسته', '۱۶۰', '۲

In [38]:
result=session.execute("select * from mydb.total_interval where year=1400 and month=4 and day=28 and hour=14") #
ids_list=[]
for row in result:
    ids_list.append(row.id)
    print(row)

Row(year=1400, month=4, day=28, hour=14, minute=40, id=327959563)
Row(year=1400, month=4, day=28, hour=14, minute=40, id=327959597)
Row(year=1400, month=4, day=28, hour=14, minute=41, id=327959661)
Row(year=1400, month=4, day=28, hour=14, minute=41, id=327959667)
Row(year=1400, month=4, day=28, hour=14, minute=41, id=327959709)
Row(year=1400, month=4, day=28, hour=14, minute=41, id=327959717)
Row(year=1400, month=4, day=28, hour=14, minute=43, id=327959859)
Row(year=1400, month=4, day=28, hour=14, minute=43, id=327959981)
Row(year=1400, month=4, day=28, hour=14, minute=44, id=327960011)
Row(year=1400, month=4, day=28, hour=14, minute=44, id=327960124)
Row(year=1400, month=4, day=28, hour=14, minute=44, id=327960132)
Row(year=1400, month=4, day=28, hour=14, minute=45, id=327960164)
Row(year=1400, month=4, day=28, hour=14, minute=45, id=327960279)
Row(year=1400, month=4, day=28, hour=14, minute=45, id=327960318)
Row(year=1400, month=4, day=28, hour=14, minute=45, id=327960322)
Row(year=1

In [39]:
res = elasticSearch.search(index="twitter_bors", body={"query": {"ids": { 'values': ids_list}}})
print("Got %d Hits:" % res['hits']['total']['value'])
for hit in res['hits']['hits']:
    print(hit['_source'])

Got 18 Hits:
{'id': '327959597', 'sendTime': '2021-07-19T10:10:38Z', 'sendTimePersian': '1400/04/28 14:40', 'senderName': 'Yass', 'senderUsername': 'myass', 'senderProfileImage': 'default', 'content': '#برکت نمادهای برتر ورود و خروج پول\u200cحقیقی در تاریخ ۲۸ تیرماه', 'type': 'twit', 'imageUid': '09823731-bf7a-4022-ab25-5d52818ac067', 'mediaContentType': 'image/jpeg', 'scoredPostDate': '1626689515293', 'finalPullDatePersian': '', 'hashtags': ['برکت'], 'links': [], 'keywords': ['نمادهای', 'ورود', 'پول\u200cحقیقی', '۲۸']}
{'id': '327959563', 'sendTime': '2021-07-19T10:10:24Z', 'sendTimePersian': '1400/04/28 14:40', 'senderName': 'roham', 'senderUsername': 'ramgol', 'senderProfileImage': '4e660afd-ea12-44ad-9d44-f7bd555d0bc6', 'content': '#خودرو با قیمت حدود ۲۵۰ باز میشه و بعدش تا حدود ۱۶۰ تومن ریزش داره مجددا رشد اما آهسته و پیوسته', 'type': 'twit', 'scoredPostDate': '1626689424655', 'finalPullDatePersian': '', 'hashtags': ['خودرو'], 'links': [], 'keywords': ['مجددا', 'پیوسته', '۱۶۰', '۲

# Finding Statistics using Cassandra

In [44]:
result=session.execute("select count(*) from mydb.total_interval where year=1400 and month=4 and day=28 and hour=14 and minute>41") #
for row in result:
    print(row)

Row(count=12)


In [42]:
result=session.execute("select avg(hour) from mydb.total_interval where year=1400 and month=4 and day=28") #
for row in result:
    print(row)

Row(system_avg_hour=14)
